In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/others/julia`


In [2]:
macro tc(ex)
    quote
        try
            while true
                $(esc(ex))
            end
        catch
           println("Exiting task")
        end
    end
end

@tc (macro with 1 method)

In [3]:
@macroexpand @tc begin 
    wait(condition)
        
    println("Task 1 Running"); 
    # sleep(2); 

    counter+=1
    println("Counter: $counter"); 

    end

quote
    #= In[2]:3 =#
    try
        #= In[2]:4 =#
        while true
            #= In[2]:5 =#
            begin
                #= In[3]:2 =#
                wait(condition)
                #= In[3]:4 =#
                println("Task 1 Running")
                #= In[3]:7 =#
                counter += 1
                #= In[3]:8 =#
                println("Counter: $(counter)")
            end
            #= In[2]:6 =#
        end
    catch
        #= In[2]:8 =#
        Main.println("Exiting task")
    end
end

In [4]:
function task1_func(condition)
    counter = 0

    @tc begin 
    wait(condition)
        
    println("Task 1 Running"); 
    # sleep(2); 

    counter+=1
    println("Counter: $counter"); 

    end
end

function task2_func(condition)
    counter = 0

    @tc begin 
    wait(condition)
        
    println("Task 2 Running"); 
    # sleep(2); 

    counter+=1
    println("Counter: $counter"); 
        
    end
end

task2_func (generic function with 1 method)

In [5]:
c = Condition()

task_1 = @task task1_func(c)
task_2 = @task task2_func(c)

Task (runnable) @0x0000000173f447f0

In [6]:
schedule(task_1)
schedule(task_2)
# schedule(block_task)

Task (runnable) @0x0000000173f447f0

In [7]:
notify(c)

Task 1 Running


2

Counter: 1
Task 2 Running
Counter: 1


In [8]:
schedule(task_1, 0; error=true)
schedule(task_2, 0; error=true)

Exiting task


Task (done) @0x0000000173f447f0

Exiting task


# Cost of task switching 

## With condition var

In [39]:
function task3_func(condition, sum_var)
    counter = 0
    
    @tc begin 
    wait(condition)

    sum_var = sum(1:1000)
    # @show sum_var

    counter+=1
    # println("Counter: $counter"); 

    notify(condition)
    end
end

c = Condition()

sum_var = 0
task_3 = @async task3_func(c, sum_var)

Task (runnable) @0x000000010e57acb0

In [37]:
for i=1:10
    notify(c)
    wait(c)
end

In [40]:
@btime begin for i=1:10
    notify(c)
    wait(c)
end

end

  280.136 μs (0 allocations: 0 bytes)


In [38]:
schedule(task_3, 0; error=true)

Exiting task


Task (done) @0x000000010e5786a0

## Without task, condition var

In [34]:
function task4_func(sum_var)
 
    sum_var = sum(1:1000)
end

sum_var = 0

0

In [35]:
@btime begin for i=1:10
    task4_func(sum_var)
end

end

  1.655 ns (0 allocations: 0 bytes)
